In [1]:
import os
import sys
sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

/home/yamada/.local/share/virtualenvs/twichAI-ydQv36PI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
def clean_text(text):
    text_ = neologdn.normalize(text)
    text_ = re.sub(r'\([^\)]*\)', "", text_)
    # text_ = re.sub(r'\d+', "0", text_)
    text_  = "".join( [m.normalized_form() if m.part_of_speech()[0]=="名詞" else m.surface() for m in tokenizer_obj.tokenize(text_, tmode)] )
    text_ = text_.replace("??", "?")
    return text_


In [28]:
def load_utt_ntt():
    ntt_path = "../../corpus/NTT/"
    utt_list = []

    for file_ in os.listdir(ntt_path):
        if not "json" in file_:
            continue 
        with open(ntt_path+file_, "r",  encoding="utf-8") as f:
            convs = json.load(f)
            for did in convs["convs"]:
                dids = list( did.keys() )[0]
                conv = did[dids]
                # conv = did[dids][3::3]
                utt_list.append( [ clean_text(utt)  for utt in conv])
    
    print(len(utt_list))
    return utt_list

In [29]:
ntt_utt = load_utt_ntt()

25015


In [30]:
def format_seq(conv):
    if len(conv)%2 != 0:
        conv = conv[:-1]
    return conv

def make_pair(convs):
    pair_list = []

    for conv in convs:
        # pair_list += np.array(format_seq(conv)).reshape(-1, 2).tolist()
        for i in range(len(conv)-2):
            pair_list.append(conv[i:i+3])
    
    return pair_list

In [31]:
pair_list = make_pair(ntt_utt)

In [32]:
len(pair_list)

91747

In [33]:
pair_list

[['こんにちは。お元気ですか?', 'はい、元気です。広告代理店での仕事が忙しいですが。', 'お疲れ様です。私は介護福祉士をしています。'],
 ['はい、元気です。広告代理店での仕事が忙しいですが。',
  'お疲れ様です。私は介護福祉士をしています。',
  '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
 ['お疲れ様です。私は介護福祉士をしています。',
  '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
  'いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。'],
 ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
  'いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。',
  '北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます'],
 ['いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。',
  '北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます',
  '北海道は、美味しい物多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。'],
 ['北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます',
  '北海道は、美味しい物多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。',
  '東京ですか。東京には華やかな場所が多いですよね'],
 ['北海道は、美味しい物多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。',
  '東京ですか。東京には華やかな場所が多いですよね',
  '田舎生まれの私にとって、とても魅力的な場所です。'],
 ['東京ですか。東京には華やかな場所が多いですよね',
  '田舎生まれの私にとって、とても魅力的な場所です。',
  '私にとっても東京は魅力的な場所ですね。お休みの日は何をされていますか?私は写真が趣味です。'],
 ['田舎生まれの私にとって、とても魅力的な場所です。',
  '私にとっても東京は魅力的

In [34]:
out_path = "../../corpus/novel_formated/"
corpus_name = "ntt_segment2.tsv"
import csv

In [35]:
with open(out_path+corpus_name, "w") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerows(pair_list)